In [51]:
import psycopg2
import ast
import json
import os
import tqdm

from utils import get_db_params

db_params = get_db_params()
conn = psycopg2.connect(**db_params)
cursor = conn.cursor()

In [6]:
def print_from_cursor(cursor):
    rows = cursor.fetchall()
    if len(rows) == 0:
        print("<!> No rows found")

    for row in rows:
        print(row)
    print()

In [46]:
# Get 5 random examples for each table ingredients, categories, recipes

cursor.execute("SELECT * FROM ingredients ORDER BY RANDOM() LIMIT 5")
print("5 random ingredients:")
print_from_cursor(cursor)

cursor.execute("SELECT * FROM categories ORDER BY RANDOM() LIMIT 5")
print("5 random categories:")
print_from_cursor(cursor)

cursor.execute("SELECT NAME FROM recipes ORDER BY RANDOM() LIMIT 5")
print("5 random recipes:")
print_from_cursor(cursor)



5 random ingredients:
(1458, 'sel marin a grain fin')
(600, 'travers de porc')
(919, 'sirop de mure')
(1518, 'eau pour couvrir')
(1394, 'sel, ou selon le gout')

5 random categories:
(10, 'cuisine francaise')
(9, 'viandes rouges')
(5, 'gouters')
(12, 'fruits de mer & crustaces')
(8, 'preparation et sauce')

5 random recipes:
('clementines confites',)
('raviolis au potiron et jambon fume',)
('dos de cabillaud a la chermoula',)
('penne aux legumes verts et ricotta',)
('sopapillas (beignets mexicains)',)



In [60]:
# Get 10 random examples of recipes with ingredients 

query = """
SELECT DISTINCT r.name
FROM recipes AS r
JOIN recipeingredients AS ri ON r.id = ri.recipeid
WHERE ri.ingredientid = (SELECT ingredientid FROM ingredients WHERE name = 'bacon')
LIMIT 4 ;
"""

cursor.execute(query)
print("4 recipes with bacon:")
print_from_cursor(cursor)


4 recipes with bacon:
("bagel d'hiver",)
('bouchees croustillantes bacon et avocat',)
("bouchees de mac'n'cheese au bacon",)
('burger au bacon et oignons confits',)



In [56]:
#strict list-based recipe search

ingredient_list = ['bacon', 'oeuf', 'tomate']

placeholders = ', '.join(['%s'] * len(ingredient_list))
query = f"""
    SELECT DISTINCT r.name
    FROM recipes AS r
    JOIN recipeingredients AS ri ON r.id = ri.recipeid
    WHERE ri.ingredientid IN (
        SELECT ingredientid FROM ingredients WHERE name IN ({placeholders})
    )
    GROUP BY r.name
    HAVING COUNT(DISTINCT ri.ingredientid) = {len(ingredient_list)};
    """

cursor.execute(query, ingredient_list)
print("recipes with all of a list (here : bacon, eggs):")
print_from_cursor(cursor)

recipes with all of a list (here : bacon, eggs):
("petit dej a l'anglaise",)
('salade cobb',)
('sucrines sauce cesar, bacon grille et oeuf mollet',)
('tortilla a la tomate, lard et oeuf sur le plat',)



In [59]:
#list-based recipe search with "wiggle" room

ingredient_list = ['bacon', 'oeuf', 'tomate', 'cheddar']
n_wiggle = 1

placeholders = ', '.join(['%s'] * len(ingredient_list))
query = f"""
    SELECT DISTINCT r.name
    FROM recipes AS r
    JOIN recipeingredients AS ri ON r.id = ri.recipeid
    WHERE ri.ingredientid IN (
        SELECT ingredientid FROM ingredients WHERE name IN ({placeholders})
    )
    GROUP BY r.name
    HAVING COUNT(DISTINCT ri.ingredientid) >= {len(ingredient_list) - n_wiggle};
    """

cursor.execute(query, ingredient_list)
print("recipes with all but one element of the list [bacon, eggs, tomato, cheddar]:")
print_from_cursor(cursor)

recipes with all but one element of the list [bacon, eggs, tomato, cheddar]:
('canneles bacon cheddar',)
('egg muffin bacon, oignon rouge et cheddar',)
("petit dej a l'anglaise",)
("poivron farci a l'omelette",)
('salade cobb',)
('sucrines sauce cesar, bacon grille et oeuf mollet',)
('texas burger',)
('tortilla a la tomate, lard et oeuf sur le plat',)

